In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mikoaro/distracteddriver")

print("Path to dataset files:", path)


100%|██████████| 4.01G/4.01G [02:19<00:00, 30.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mikoaro/distracteddriver/versions/1


In [ ]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
import torchvision.models as models


In [ ]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
warnings.filterwarnings('ignore')

In [ ]:
train_path='/root/.cache/kagglehub/datasets/mikoaro/distracteddriver/versions/1/distracted-driver-detection/train'
test_path='/root/.cache/kagglehub/datasets/mikoaro/distracteddriver/versions/1/distracted-driver-detection/test'

In [ ]:
# Parameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 10
TRAIN_DIR = train_path
TEST_DIR = test_path
split_ratio=0.2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),  # 480x480
    transforms.RandomHorizontalFlip(),  #for increasing diversity
    transforms.RandomRotation(15),
    transforms.ToTensor(),      #converting to tensor data
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])


In [ ]:
images = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)

In [ ]:
images.classes

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [ ]:
images.class_to_idx

{'c0': 0,
 'c1': 1,
 'c2': 2,
 'c3': 3,
 'c4': 4,
 'c5': 5,
 'c6': 6,
 'c7': 7,
 'c8': 8,
 'c9': 9}

In [ ]:
type(images)

torchvision.datasets.folder.ImageFolder

In [ ]:
images[0]

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

In [ ]:
len(images)

22424

In [ ]:

num_total = len(images)
num_val = int(split_ratio* num_total)
num_train = num_total - num_val

# Split the dataset into training and validation sets
train_data, test_data = random_split(images, [num_train, num_val])




In [ ]:

print(f"Number of training samples: {len(train_data)}")
print(f"Number of validation samples: {len(test_data)}")

Number of training samples: 17940
Number of validation samples: 4484


In [ ]:
# DataLoaders for training and validation
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Move model to GPU if available
model = model.to(DEVICE)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 150MB/s]


In [ ]:
def train_model(model, train_loader, test_loader, epochs=2, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        #setting the model to training mode
        model.train()
        #loss of current epoch
        curr_loss = 0
        correct_pred=0
        total_pred=0

        for images, classes in tqdm(train_loader, desc="Training", leave=False):
            #clearing the gradients
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, classes )
            loss.backward()
            optimizer.step()

            curr_loss += loss.item()

            #the maximum value along the label dimension
            value, predicted = torch.max(outputs, 1)
            total_pred += classes .size(0)
            correct_pred += (predicted == classes).sum().item()

        train_accuracy = 100 * correct_pred / total_pred
        avg_loss=curr_loss/len(train_loader)

        print(f'Epoch {epoch+1}, Average Loss: {avg_loss}, Accuracy: {train_accuracy}')

        # evaluation mode
        model.eval()
        #disable grad computation
        with torch.no_grad():

            correct_test_pred=0
            total_test_pred=0
            test_loss = 0

            for images, classes in test_loader:
                outputs = model(images)
                test_loss += criterion(outputs, classes).item()
                value, predicted = torch.max(outputs, 1)
                total_test_pred += classes.size(0)
                correct_test_pred += (predicted == classes).sum().item()

            test_accuracy = 100 * correct_test_pred / total_test_pred

            print(f'Validation Accuracy: {test_accuracy}')

In [ ]:
train_model(model,train_loader,test_loader)

Epoch 1, Average Loss: 0.5216503134810776, Accuracy: 83.13823857302118
Validation Accuracy: 93.53256021409456


Training:  81%|████████▏ | 456/561 [3:05:22<42:09, 24.09s/it]

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'distracted_driver_model2.pth')


In [ ]:
# Load the model
model.load_state_dict(torch.load('distracted_driver_model2.pth'))
model.eval()

# Predict on a single image
from PIL import Image

def predict(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return predicted.item()

image_path = '/test_image.jpeg'
class_idx = predict(image_path, model, test_transform)
class_name = images.classes[class_idx]
print(f"Predicted Class: {class_name}")
